# PME3481 - Controle e Aplicações

## Controlabilidade e Observabilidade

Supondo um SLIT, o código permite avaliar se é ou não controlável a partir das matrizes A e B, avaliando o gramiano, tal qual utilizando as matrizes A e C, avaliando a observabilidade.

In [4]:
import numpy as np
import control

As duas funções que realizam a análise de controlabilidade e observabilidade do sistema

In [5]:
def is_controllable(A, B):
    """
    Check if a system defined by its state matrix A and input matrix B is controllable.
    """
    return np.linalg.matrix_rank(control.ctrb(A, B)) == A.shape[0], control.ctrb(A, B)

In [6]:
def is_observable(A, C):
    """
    Check if a system defined by its state matrix A and output matrix C is observable.
    """
    return np.linalg.matrix_rank(control.obsv(A, C)) == A.shape[0], control.obsv(A, C)

Abaixo, as matrizes do sistema definidas

In [7]:
Xcm = 1.052
Xpe = 0.261
L = 3.304
Df = 0.127
Tlin = 230.57
Izz = 15.07
M = 23.545
g = 9.8055
dar = 1.091
Cd = 0.432
Vxlin = -10.0
Vvlin = 2.2

In [8]:
# X = [theta,xp,yp,thetap]
# Xp = [thetap,xpp,ypp,thetapp]
A = np.array([[0, 0, 0, 1],
              [(1/M)*((1/2)*dar*Cd*Vxlin**2*L*Df + (1/2)*dar*Cd*L*Df*Vvlin**2), (1/M)*(Vxlin *dar*Cd*((Df/2)**2*np.pi)), 0, 0],
              [(1/M)*(-M*g+(1/2)*Vxlin**2*dar*Cd*(np.pi*(Df/2)**2) - (1/2)*dar*Cd*(np.pi*(Df/2)**2)*Vvlin**2), 0, 0, 0],
              [(1/Izz)*((1/2)*Vxlin**2*dar*Xpe*Cd*(np.pi*(Df/2)**2)- (1/2)*dar*Cd*Xpe*(np.pi*(Df/2)**2)*Vvlin**2), 0, 0, 0]])

# u = [T,tau]

# B = np.array([[0, 0, 0],
#               [1/M, 0, 0],
#               [0, -Tlin/M, - (1/2)*dar*Cd*L*Df*2*Vvlin],
#               [0, -Tlin*Xcm/Izz, - (1/2)*dar*Cd*Xpe*L*Df*2*Vvlin]]) 

B = np.array([[0, 0, 0],
              [1/M, 0, 0],
              [0, -Tlin/M, 0],
              [0, -Tlin*Xcm/Izz, 0]])

E = np.array([[0],
              [0],
              [ (- (1/2)*dar*Cd*L*Df*2*Vvlin)/(M*Izz)],
              [(- (1/2)*dar*Cd*Xpe*L*Df*2*Vvlin)/(M*Izz)]])

C = np.array([[ 0, 1, 0, 0],
              [ 0, 0, 1, 0],])

D = np.array([[ 0, 0, 0],
              [ 0, 0, 0]])

cont = control.StateSpace(A, B, C, D)

sys = control.ss(A, B, C, D)
sys

<LinearIOSystem:sys[3]:['u[0]', 'u[1]', 'u[2]']->['y[0]', 'y[1]']>

In [9]:
is_controllable_result, control_matrix = is_controllable(sys.A, sys.B)
if is_controllable_result:
    print("System is controllable.")
else:
    print("System is not controllable.")

print("controlability Matrix")
print(control_matrix)

is_observable_result, observe_matrix = is_observable(sys.A, sys.C)
if is_observable_result:
    print("System is observable.")
else:
    print("System is not observable.")

print("Observability Matrix")
print(observe_matrix)

System is controllable.
controlability Matrix
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -1.60955302e+01  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -7.91886247e-02  0.00000000e+00]
 [ 4.24718624e-02  0.00000000e+00  0.00000000e+00 -1.07698197e-04
   0.00000000e+00  0.00000000e+00  2.73096137e-07 -7.08689284e+00
   0.00000000e+00 -6.92504632e-10  1.79706171e-02  0.00000000e+00]
 [ 0.00000000e+00 -9.79273731e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  1.57630527e+02
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -1.60955302e+01  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -7.91886247e-02
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]
System is observable.
Observability Matrix
[[ 0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+